ENVIRON

In [1]:
import networkx as nx
import pandas as pd
import os
import numpy as np
import seaborn as sns
from tqdm import tqdm, trange
from itertools import combinations

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag

In [3]:
tqdm.pandas()

In [4]:
lemmatizer = WordNetLemmatizer()
stopword_list = stopwords.words('english')

In [5]:
def flatten(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i

In [6]:
def morphs(text, noun = True, verb = False, adjective = False, adverb = False):
    poses = pos_tag(text, tagset = 'universal')
    filters = []

    if noun:
        filters.append('NOUN')
    if verb:
        filters.append('VERB')
    if adjective:
        filters.append('ADJ')
    if adverb:
        filters.append('ADV')

    return [pos[0] for pos in poses if pos[1] in filters]

In [7]:
for index, model_option in enumerate(os.listdir('./datasets/')):
    # INDEXING
    print(f'[{index}/{len(os.listdir("./datasets/"))}] - {model_option}')
    if model_option == 'original' or model_option == 'tfidf':
        continue

    # ------------------------------

    # DATASET - ROBLOX
    roblox1_df = pd.read_csv(f'./datasets/{model_option}/roblox1.csv', index_col = 0, low_memory = False)
    roblox2_df = pd.read_csv(f'./datasets/{model_option}/roblox2.csv', index_col = 0, low_memory = False)
    roblox3_df = pd.read_csv(f'./datasets/{model_option}/roblox3.csv', index_col = 0, low_memory = False)
    roblox4_df = pd.read_csv(f'./datasets/{model_option}/roblox4.csv', index_col = 0, low_memory = False)
    roblox5_df = pd.read_csv(f'./datasets/{model_option}/roblox5.csv', index_col = 0, low_memory = False)

    df = pd.concat([roblox1_df, roblox2_df, roblox3_df, roblox4_df, roblox5_df]).dropna(subset = ['keybert_keywords']).reset_index(drop = True)

    # PREPROCESS
    df['keybert_keywords'] = df['keybert_keywords'].progress_apply(eval)                                                    # str2list
    df['keywords'] = df['keybert_keywords'].progress_apply(lambda x : list(flatten(x))[0::2] if len(x) > 0 else '')         # select only keywords
    df['keywords'] = df['keywords'].progress_apply(lambda x : [y for y in x if y.isalpha()])                                # exclude if numeric
    df['keywords'] = df['keywords'].progress_apply(lambda x : [lemmatizer.lemmatize(y) for y in x])                         # lemmatize
    df['keywords'] = df['keywords'].progress_apply(lambda x : list(set(x)))                                                 # drop duplicate
    df['keywords'] = df['keywords'].progress_apply(lambda x : [y for y in x if len(y) >= 3 and len(y) <= 15])               # 3 <= len(keyword) <= 15
    df['keywords'] = df['keywords'].progress_apply(morphs, noun = True, verb = True, adjective = False, adverb = False)     # select noun

    # GRAPH FORMULATION - ROBLOX
    G = nx.MultiGraph()
    counts = df['keywords'].explode().reset_index(drop = True).reset_index().groupby('keywords').count()['index']
    counts = counts[counts > 10]

    # ADD NODES
    for item in tqdm(counts.index):
        G.add_node(item, weight = counts[item])

    # ADD EDGES
    for keywords in tqdm(df['keywords']):
        keyword_selected = [keyword for keyword in keywords if keyword in counts.index]

        pairs = list(combinations(keyword_selected, 2))
        pairs = [(pair[0], pair[1]) if pair[0] < pair[1] else (pair[1], pair[0]) for pair in pairs]
        
        G.add_edges_from(pairs)

    # MULTI to SINGLE - ROBLOX
    H = nx.Graph()

    # MULTI to SINGLE - ADD NODES
    for item in tqdm(counts.index):
        H.add_node(item, weight = counts[item])

    # MULTI to SINGLE - ADD EDGES
    for u, v, data in tqdm(G.edges(data = True)):
        w = data['weight'] if 'weight' in data else 1.0
        if H.has_edge(u, v):
            H[u][v]['weight'] += w
        else:
            H.add_edge(u, v, weight=w)

    # WRITE GRAPH
    nx.write_graphml_lxml(H, f'./graph/roblox-{model_option}.graphml')

    # ------------------------------

    # DATASET - ZEPETO
    df = pd.read_csv(f'./datasets/{model_option}/zepeto.csv', index_col = 0, low_memory = False)

    # PREPROCESS
    df['keybert_keywords'] = df['keybert_keywords'].progress_apply(eval)                                                    # str2list
    df['keywords'] = df['keybert_keywords'].progress_apply(lambda x : list(flatten(x))[0::2] if len(x) > 0 else '')         # select only keywords
    df['keywords'] = df['keywords'].progress_apply(lambda x : [y for y in x if y.isalpha()])                                # exclude if numeric
    df['keywords'] = df['keywords'].progress_apply(lambda x : [lemmatizer.lemmatize(y) for y in x])                         # lemmatize
    df['keywords'] = df['keywords'].progress_apply(lambda x : list(set(x)))                                                 # drop duplicate
    df['keywords'] = df['keywords'].progress_apply(lambda x : [y for y in x if len(y) >= 3 and len(y) <= 15])               # 3 <= len(keyword) <= 15
    df['keywords'] = df['keywords'].progress_apply(morphs, noun = True, verb = True, adjective = False, adverb = False)     # select noun
    df['keywords'] = df['keywords'].progress_apply(lambda x : [y for y in x if y not in stopword_list])

    # GRAPH FORMULATION - ZEPETO
    G = nx.MultiGraph()
    counts = df['keywords'].explode().reset_index(drop = True).reset_index().groupby('keywords').count()['index']
    counts = counts[counts > 10]

    # ADD NODES
    for item in tqdm(counts.index):
        G.add_node(item, weight = counts[item])

    # ADD EDGES
    for keywords in tqdm(df['keywords']):
        keyword_selected = [keyword for keyword in keywords if keyword in counts.index]

        pairs = list(combinations(keyword_selected, 2))
        pairs = [(pair[0], pair[1]) if pair[0] < pair[1] else (pair[1], pair[0]) for pair in pairs]
        
        G.add_edges_from(pairs)

    # MULTI to SINGLE - ZEPETO
    H = nx.Graph()

    # MULTI to SINGLE - ADD NODES
    for item in tqdm(counts.index):
        H.add_node(item, weight = counts[item])

    # MULTI to SINGLE - ADD EDGES
    for u, v, data in tqdm(G.edges(data = True)):
        w = data['weight'] if 'weight' in data else 1.0
        if H.has_edge(u, v):
            H[u][v]['weight'] += w
        else:
            H.add_edge(u, v, weight=w)

    # WRITE GRAPH
    nx.write_graphml_lxml(H, f'./graph/zepeto-{model_option}.graphml')

[0/8] - tfidf
[1/8] - keybert-all-MiniLM-L12-v2


100%|██████████| 643637/643637 [00:01<00:00, 391112.92it/s]


[2/8] - keybert-all-distilroberta-v1


100%|██████████| 641165/641165 [00:01<00:00, 391322.72it/s]


[3/8] - keybert-all-mpnet-base-v2


...

[5/8] - keybert-multi-qa-distilbert-cos-v1


100%|██████████| 612778/612778 [00:01<00:00, 398098.86it/s]


[6/8] - keybert-multi-qa-mpnet-base-dot-v1


100%|██████████| 614529/614529 [00:01<00:00, 393691.13it/s]


[7/8] - original
